In [60]:
"""
Import Required Packages
"""
from sklearn.model_selection import train_test_split
import numpy as np
import math
import random
import pandas as pd


In [61]:
line = 0
dat = []

"""
Reading Data in List.

"""
with open('spambase.data', mode='r') as f:
    line_count = 0
    for row in f:
        l = row.strip().split(",")
        l = map(float, l)
        # print l
        dat.append(l)



In [62]:
"""
Dividing datasets by Classes which is {1:attributes and 0:attributes}
"""


def sep_class(train1):
    class_mapping = {}

    for i in range(len(train1)):
        temp = train1[i]
        cl = temp[-1]
        val = temp[:-1]

        if cl not in class_mapping:
            class_mapping[cl] = []
        class_mapping[cl].append(val)
    return class_mapping





In [63]:
""""
Finding mean and std. Deviation function for Gussian distribution.

"""

def mean(listt):
    return sum(listt) / float(len(listt))



def stdev(numbers):
    cal_mean = mean(numbers)
    variance = sum([pow(x - cal_mean, 2) for x in numbers]) / float(len(numbers) - 1)
    return math.sqrt(variance)




In [64]:
"""
Helper function to calculate mean and variance for each class (column wise) 
like '(mean,Std_deviation)' tuples of 57 features and 2 classes each which 
is 114 values.
"""
def helper(dataset):
    mean_var_dict = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)]
    return mean_var_dict



In [65]:
"""
Calculate Gaussian Density and meanwhile performed smoothing for extremely small Values Not used add one smoothing 
because  most of thethe integers Values of datasets are smaller than 1 which would might distort the Model
"""

def density_Prob(x, mean, stdev):
    if (mean == 0.0 or stdev == 0.0):
        mean += 0.0000000001                                # Smoothing for some very small value
        
        stdev += 0.00000000001                              # Smoothing for some very small value
    exponent = math.exp(-(math.pow(x - mean, 2) / (2 * math.pow(stdev, 2))))
    return (1 / (math.sqrt(2 * math.pi) * stdev)) * exponent






In [66]:
"""
Finding and multiplying the pdf(each attribute of test input) to compare the class

"""

def calculateClassProbabilities(mean_var_dict, inputt):
    probabilities = {}
    for classValue, value in mean_var_dict.iteritems():
        probabilities[classValue] = 1
        for i in range(len(value)):
            mean, stdev = value[i]
            x = inputt[i]
            probabilities[classValue] *= density_Prob(x, mean, stdev)
    return probabilities



In [67]:


"""
Choosing the best label by comparing class1_prob < class2_prob tag class2
"""


def predict(summaries, inputt):
    inputt=inputt[:-1]
    probabilities = calculateClassProbabilities(summaries, inputt)
    bestLabel, bestProb = None, -1
    
    """
    Finding Best class Probability by comparing for class
    """
    for classValue, probability in probabilities.iteritems():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classValue
    return bestLabel

"""
storing all prdicted Values for each class in list named Predictions
"""
def Calculate_Predictions(summaries, testSet):
    predictions = []
    for i in range(len(testSet)):
        result = predict(summaries, testSet[i])
        predictions.append(result)
    return predictions



"""
To calculate False positive , False Negative and Error tags

"""
def Calculate_Accuracy(testSet, predictions):
    correct = 0
    fp=0
    totalfp=0
    totalfn=0
    fn=0
    for i in range(len(testSet)):
        
        """
            Counting False Positive
        """

        if testSet[i][-1] == 0:
            if testSet[i][-1] != predictions[i]:
                fp=fp+1
            totalfp=totalfp+1
            
        """
        Counting False Negative
        """
        if testSet[i][-1] == 1:
            if testSet[i][-1] != predictions[i]:
                fn = fn + 1
            totalfn = totalfn + 1
        """
            Calculating accuracy for Error Rate
        """



        if testSet[i][-1] == predictions[i]:
            correct += 1

    return (correct / float(len(testSet))) * 100.0,(fp/float(totalfp)),(fn/float(totalfn))




In [68]:

"""
K-Folds Cross Validation k=5

"""

random.shuffle(dat)
k = 5

part = math.ceil(len(dat) / 5)
itemp = 0
train1=[]
test1=[]
sum_error=0
Elist=[]
Flist=[]
Nlist=[]
for i in range(0, 5):

    l=0

    train1=[]
    test1=[]


    testfig = []
    trainfig = []
    part = int(part)
    l = int((i + 1) * part)

    testfig = dat[itemp:l]

    trainfig = [item for item in dat if item not in testfig]

    itemp = l

    """
    Splitting Train and Test by one Part k for test data and rest as training data
    """
    train1,test1=trainfig,testfig  

    class_mapping = sep_class(train1)
    
    """
    (Mean,Variance) Dictionary by class 0 and 1 mean_var_dict={} 
    """
    mean_var_dict = {}
    for classValue, instances in class_mapping.iteritems():
        mean_var_dict[classValue] = helper(instances)

    predictions = Calculate_Predictions(mean_var_dict, test1)

    accuracy,false_positve,false_negative = Calculate_Accuracy(test1, predictions)

    error=100-accuracy

    sum_error=error+sum_error    # add errors to find average

    #print "Error rates:",error
    #print "false positive",false_positve
    #print "false negative",false_negative

    Elist.append(error)
    Flist.append(false_positve)
    Nlist.append(false_negative)

    
    
    
    


In [69]:
"""
Using Pandas Framework to form the Table

"""


Lab_list=["Average Error Rate in (%)" ,"  is" ,sum_error/5]



labels=("False Positive","False Negative", "Error Rate")


Elist=map(str,Elist)
Flist=map(str,Flist)
Nlist=map(str,Nlist)
Lab_list=map(str,Lab_list)





p=zip(Flist,Nlist,Elist)

Lab_list=tuple(Lab_list)


p.append(Lab_list)


table=pd.DataFrame.from_records(p,columns=labels)

table.index.name="Fold"
table.index+=1


print table

print "Accuracy of the Model with last fold is ",accuracy,"%"

                 False Positive   False Negative     Error Rate
Fold                                                           
1                0.248695652174  0.0550724637681  17.6086956522
2                 0.27255985267  0.0742705570292  19.1304347826
3                0.284946236559  0.0303867403315  18.4782608696
4                0.264705882353  0.0263157894737  17.6086956522
5               0.0994371482176   0.108527131783  10.3260869565
6     Average Error Rate in (%)               is  16.6304347826
Accuracy of the Model with last fold is  89.6739130435 %
